In [0]:
import pandas as pd

tweets = pd.read_csv('Airline_tweets.csv', encoding="ISO-8859-1")

In [79]:
tweets.head()

,sentiment,tweet
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [80]:
import re
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
def removeUrls(text):
  tweetout = re.sub(r'@[a-zA-Z0-9]+','',text)
  re.sub('https?://[a-zA-Z0-9./]+','',tweetout)
  return tweetout

def removeNonAlphaNumericCharac(text):
  text_out = "".join([i for i in text if i not in string.punctuation])
  return text_out

In [0]:
tweets['tweetNoMenUrls'] = tweets['tweet'].apply(lambda x:removeUrls(x))

In [0]:
tweets['tweetNoPuncs'] = tweets['tweetNoMenUrls'].apply(lambda x:removeNonAlphaNumericCharac(x))

In [0]:
def tokenize(text):
  tokens = re.split('\W+', text)
  return tokens

In [0]:
tweets['tokens'] = tweets['tweetNoPuncs'].apply(lambda x:tokenize(x))

In [0]:
import nltk
ps = nltk.PorterStemmer()
def stemming(text):
  out_text=[ps.stem(word) for word in text]
  return out_text

tweets['stemwords'] = tweets['tokens'].apply(lambda x:stemming(x))

In [87]:
import nltk
nltk.download('wordnet')
ps = nltk.WordNetLemmatizer()
def lemmatizeword(text):
  out_text=[ps.lemmatize(word) for word in text]
  return out_text

tweets['lemmawords'] = tweets['tokens'].apply(lambda x:lemmatizeword(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [88]:
tweets.head()

,sentiment,tweet,tweetNoMenUrls,tweetNoPuncs,tokens,stemwords,lemmawords
0,neutral,@VirginAmerica What @dhepburn said.,What said.,What said,"[, What, said]","[, what, said]","[, What, said]"
1,positive,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experien...,plus youve added commercials to the experienc...,"[, plus, youve, added, commercials, to, the, e...","[, plu, youv, ad, commerci, to, the, experi, t...","[, plus, youve, added, commercial, to, the, ex..."
2,neutral,@VirginAmerica I didn't today... Must mean I n...,I didn't today... Must mean I need to take an...,I didnt today Must mean I need to take anothe...,"[, I, didnt, today, Must, mean, I, need, to, t...","[, I, didnt, today, must, mean, I, need, to, t...","[, I, didnt, today, Must, mean, I, need, to, t..."
3,negative,@VirginAmerica it's really aggressive to blast...,"it's really aggressive to blast obnoxious ""en...",its really aggressive to blast obnoxious ente...,"[, its, really, aggressive, to, blast, obnoxio...","[, it, realli, aggress, to, blast, obnoxi, ent...","[, it, really, aggressive, to, blast, obnoxiou..."
4,negative,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,and its a really big bad thing about it,"[, and, its, a, really, big, bad, thing, about...","[, and, it, a, realli, big, bad, thing, about,...","[, and, it, a, really, big, bad, thing, about,..."


In [0]:
import itertools
stemwords = len(set(list(itertools.chain.from_iterable(tweets['stemwords']))))
lemmawords = len(set(list(itertools.chain.from_iterable(tweets['lemmawords']))))


In [90]:
print(stemwords)
print(lemmawords)

13565
18138


In [91]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
stopwords = nltk.corpus.stopwords.words('english')

In [0]:
def removeStopwords(tokenizedlist):
  text_out = [word for word in tokenizedlist if word not in stopwords]
  return text_out

In [0]:
tweets['seconddataset'] = tweets['stemwords'].apply(lambda x:removeStopwords(x))

In [0]:
#remove low frequencies words

In [0]:
listofuniquewords = itertools.chain.from_iterable(tweets['seconddataset'])

In [0]:
def join_tokens(tokens):
  doc = " ".join([word for word in tokens if not word.isdigit()])
  return doc

In [0]:
tweets['thirddataset'] = tweets['seconddataset'].apply(lambda x:join_tokens(x))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(listofuniquewords)
countvector = cv.fit_transform(tweets['thirddataset'])
countvectordf = pd.DataFrame(countvector.toarray())
countvectordf.columns = cv.get_feature_names()

In [100]:
countvectordf.head()

,021mbp,10000lb,1000cost,1000p,1000pm,1001pm,1005am,1005pm,1007p,100pm,1014am,1015am,1020pm,1030a,1030pm,1045pm,1050am,1051pm,1055pm,10a,10am,10d,10f,10hour,10hr,10m,10min,10minut,10p,10pm,10th,10voucherwhatajok,10x,10x9x17,10yr,1100pm,1110p,1115p,1120am,1121p,...,ûthat,ûtri,ûwell,ûª,ûªall,ûªd,ûªl,ûªll,ûªm,ûªre,ûªt,ûªve,ûï,ûïear,ûïgay,ûïinconveni,ûïit,ûïonli,ûïsouthwest,ûïthank,ûïthat,ûïtide,ûïwe,ûïyouv,ûò,ûó,ûógt,ûóit,ûóive,ûómi,ûónoth,ûóthe,ûóuse,ûówhi,ûù,ûü,üi,üsfo,üù,ýã
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#tweets['sentiment'] = le.fit_transform(tweets['sentiment'])
#tweets['sentiment'].value_counts()

In [0]:
from sklearn.model_selection import train_test_split
X = countvectordf
y = tweets['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=0)

In [103]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model = rf.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
preds = model.predict(X_test)

In [108]:
from sklearn.metrics import classification_report
print((classification_report(y_test, preds)))


              precision    recall  f1-score   support

    negative       0.81      0.91      0.86      1870
     neutral       0.59      0.50      0.54       614
    positive       0.75      0.52      0.61       444

    accuracy                           0.76      2928
   macro avg       0.71      0.64      0.67      2928
weighted avg       0.75      0.76      0.75      2928



In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(tweets['thirddataset']).toarray()

#countvectordf = pd.DataFrame(processed_features)
#countvectordf.columns = processed_features.get_feature_names()

In [0]:
from sklearn.model_selection import train_test_split
X = processed_features
y = tweets['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=0)

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, random_state=0)
model = rf.fit(X_train, y_train)


In [0]:
preds = model.predict(X_test)

In [125]:
from sklearn.metrics import classification_report
print((classification_report(y_test, preds)))


              precision    recall  f1-score   support

    negative       0.80      0.92      0.86      1870
     neutral       0.63      0.45      0.53       614
    positive       0.74      0.55      0.63       444

    accuracy                           0.77      2928
   macro avg       0.72      0.64      0.67      2928
weighted avg       0.76      0.77      0.75      2928

